In [4]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


#This is what we will change if we want the whole data. Right now it only runs for 44th president

for i in range(25,44):
    x = i
    absolutepath_president = f'/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[{x}]/label'
    president1 = browser.find_element_by_xpath(absolutepath_president)
    president1.click()
    time.sleep(2)




SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')

In [5]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string
import numpy as np



link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        


http = urllib3.PoolManager()
list2=[]
president_names=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    president_names.append(soup_2.find('p', {"class":"president-name"}).get_text(strip=True))
    


#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    if list2[i]:
        list3.append(str(list2[i]))
    else:
        
        #add comments for why I had to do it like this
        
        url = "http://millercenter.org" + new_link_str[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        list3.append(str(soup_2.findAll('div', {"class":"view-transcript"})))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches, 'president':president_names})

#this part is added because dataframe does not recognize [] as an empty input. 

df['speeches'].replace('[]', np.nan, inplace=True)

#now that I replaced [] with NaN, I can simply say drop all the rows with NaN elements

df.dropna(subset=['speeches'], inplace=True)

In [6]:
df.to_pickle("./speech_data.pkl")

In [7]:
#Import libraries for the function
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
import numpy as np
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from statistics import mean
import pandas as pd


def filter_subj_sentences(text: str) -> list:
    
    """Create a function using NLTK and Textblob package to filter
    sentences in 'text' variable within a range of subjectivity
    """
    
    #Tokenize text and add SentimentIntensityAnalyzer to a variable
    sent_tokenize_list = sent_tokenize(text)
    filtered_text = []
    sid = SentimentIntensityAnalyzer()
    
    #Only filters speeches with more than 50 sentences
    if len(sent_tokenize_list) > 50:
        #Filter sentences with subjectivity less than 0.2 or more than 0.85
        for sentence in sent_tokenize_list:
            ss = list(TextBlob(sentence).sentiment) #ss on list format will give us [polarity, subjectivity]
            if 0.2 < ss[1] < 0.85:
                filtered_text.append(sentence)
                
    #Else we won't filter the speech            
    else:
        filtered_text = sent_tokenize_list
    
    #This function will return a the text tokenized and filtered on a list
    return filtered_text

#TEST THE FUNCTION
#assert type(filter_subj_sentences(text)) == list

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


D:\Users\diego\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
def get_sentimental_values(sent_tokenize_list: list) -> list:
    
    """
    
    """
    
    score_compound = []
    score_neg=[]
    score_pos=[]
    sid = SentimentIntensityAnalyzer()
    for sentence in sent_tokenize_list:
        ss = sid.polarity_scores(sentence)
        score_compound.append(ss['compound'])
        score_neg.append(ss['neg'])
        score_pos.append(ss['pos'])

    len(sent_tokenize_list)
    
    score_compound_array = np.array(score_compound)
    score_pos_array = np.array(score_pos)
    score_neg_array = np.array(score_neg)

    max_pos_sent_idx = score_pos_array.argsort()[-25:][::-1]
    max_pos_sent = []
    max_pos_scr = []

    max_neg_sent_idx = score_neg_array.argsort()[-25:][::-1]
    max_neg_sent = []
    max_neg_scr = []

    for i in max_pos_sent_idx:
        max_pos_sent.append(sent_tokenize_list[i])
        max_pos_scr.append(score_pos[i])
    max_pos_sent

    for i in max_neg_sent_idx:
        max_neg_sent.append(sent_tokenize_list[i])
        max_neg_scr.append(score_neg[i])
    max_neg_sent
    #took mean
    return [mean(max_pos_scr),mean(max_neg_scr)]
#    return[max_pos_sent,max_neg_sent]

In [9]:
def sentimental_analysis(dataframe):
    
    positive_scores = []
    negative_scores = []
    compound_scores = []
    for row in dataframe['speeches']:
        
        if row != []:
            
            positive_scores.append(get_sentimental_values(filter_subj_sentences(row))[0])
            negative_scores.append(get_sentimental_values(filter_subj_sentences(row))[1])
            score = []

            for sentence in filter_subj_sentences(row):
                sid = SentimentIntensityAnalyzer()
                ss = sid.polarity_scores(sentence)
                score.append(ss['compound'])
            #took mean away
            compound_scores.append(mean(score))
        
    dataframe['positive'] = positive_scores
    dataframe['negative'] = negative_scores
    dataframe['compound'] = compound_scores
    return dataframe

In [11]:
unpickled_df = pd.read_pickle("./speech_data.pkl")

In [12]:
unpickled_df.head(5)

,dates,speeches,president
0,2016-05-15,[Hello Rutgers! (Applause.) R-U rah-rah! (A...,Barack Obama
1,2016-03-22,[Thank you. (Applause.) Muchas gracias. Tha...,Barack Obama
2,2016-01-12,"[Mr. Speaker, Mr. Vice President, Members of C...",Barack Obama
3,2015-06-26,[THE PRESIDENT: Giving all praise and honor t...,Barack Obama
4,2015-03-07,"[UDIENCE MEMBER: We love you, President Obama...",Barack Obama


In [13]:
testy = sentimental_analysis(unpickled_df)

In [21]:
testy.to_pickle("./FINAL SPEECH PRES SCORE DATA.pkl")

In [22]:
testy.tail(5)

,dates,speeches,president,positive,negative,compound
928,1802-11-03,[TO BROTHER HANDSOME LAKE: I have re...,Thomas Jefferson,0.21000,0.06136,0.357121
929,1802-01-01,"[ GENTLEMAN, The affectionate sentiments of e...",Thomas Jefferson,0.26260,0.01760,0.880960
930,1801-12-08,[ FELLOW CITIZENS OF THE SENATE AND HOUSE OF R...,Thomas Jefferson,0.21020,0.10064,0.361692
931,1801-07-12,"[ GENTLEMAN, I have received the remonstrance...",Thomas Jefferson,0.18392,0.13816,0.147983
932,1801-03-04,"[ FRIENDS AND FELLOW-CITIZENS, Called upon t...",Thomas Jefferson,0.24652,0.10032,0.322571


In [60]:
lolo = pd.read_pickle("./FINAL SPEECH PRES SCORE DATA.pkl")

In [61]:
lolo = lolo.set_index('dates')

In [62]:
lolo.head(5)

,speeches,president,positive,negative,compound
dates,,,,,
2016-05-15,[Hello Rutgers! (Applause.) R-U rah-rah! (A...,Barack Obama,0.47012,0.27448,0.228497
2016-03-22,[Thank you. (Applause.) Muchas gracias. Tha...,Barack Obama,0.38732,0.20736,0.297054
2016-01-12,"[Mr. Speaker, Mr. Vice President, Members of C...",Barack Obama,0.38668,0.28860,0.184648
2015-06-26,[THE PRESIDENT: Giving all praise and honor t...,Barack Obama,0.36152,0.18656,0.267056
2015-03-07,"[UDIENCE MEMBER: We love you, President Obama...",Barack Obama,0.25692,0.17868,0.233208


In [63]:
lolo = lolo.truncate(after=pd.Timestamp('1901-09-14')).tail()

In [55]:
lolo = lolo.reset_index()

In [64]:
lolo.tail(5)

,speeches,president,positive,negative,compound
dates,,,,,
1903-10-20,[By the President of the United States of Amer...,Theodore Roosevelt,0.16400,0.000000,0.989200
1902-12-02,[To the Senate and House of Representatives:We...,Theodore Roosevelt,0.33336,0.233800,0.319026
1902-06-13,[To the Senate and House of Representatives:I ...,Theodore Roosevelt,0.19936,0.076920,0.340781
1901-12-03,[To the Senate and House of Representatives:Th...,Theodore Roosevelt,0.34924,0.303400,0.260799
1901-09-14,[By the President of the United States of Amer...,Theodore Roosevelt,0.21000,0.099667,0.718633


In [65]:
lolo.to_pickle("./DATA.pkl")